<a href="https://colab.research.google.com/github/chiao-lyc/HW4/blob/master/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip3 install opencc-python-reimplemented

In [12]:
!pip3 install gensim

In [31]:
!pip install word2vec

     |████████████████████████████████| 42 kB 505 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for word2vec: filename=word2vec-0.11.1-py2.py3-none-any.whl size=156423 sha256=4d5810e8f70092b12825d93d922ed5a8ba04127df5c574dec0359062189d4f20
  Stored in directory: /root/.cache/pip/wheels/c9/c0/d4/29d797817e268124a32b6cf8beb8b8fe87b86f099d5a049e61
Successfully built word2vec


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
import pandas as pd
import os

filePath = []
def show_folder_content(folder_path):
    folder_content = os.listdir(folder_path)
    for item in folder_content:
        if os.path.isdir(folder_path + '/' + item):
            show_folder_content(folder_path + '/' + item)
        elif os.path.isfile(folder_path + '/' + item):
            filePath.append(folder_path + '/' + item)
        else:
            print('無法辨識：' + item)


# 顯示指定資料夾的內容
target_folder = '/content/drive/My Drive/wiki_zh'
show_folder_content(target_folder)
print("Finish", len(filePath))

Finish 1246


In [ ]:
# coding=utf-8
from opencc import OpenCC
import jieba 
import json
import re

dictionaryPath = '/content/drive/My Drive/wiki_zh/Model/dict.txt.big'
jieba.set_dictionary(dictionaryPath)
cc = OpenCC('s2t')

# 讀取停用詞
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords 

def LoadToSeg(filepath, fW):
    with open(filepath, 'r') as obj:
        for line in obj.readlines():
            dic = json.loads(line)
            temp_text = re.findall('[\u4e00-\u9fa5]', cc.convert(dic['text']))
            text = ''.join(temp_text)
            tags = jieba.lcut(text, cut_all=False)
            ThisLine = ''
            for tag in tags:
                if tag not in stopwords:  
                    if tag != '\t' and tag != '\n' and tag != '' and len(tag) > 1:  
                        ThisLine += tag  
                        ThisLine += " "  
            fW.write((ThisLine+"\n").encode('utf-8'))
stopwords = stopwordslist('/content/drive/My Drive/wiki_zh/Model/StopWords.txt') 
fileSegWordDonePath ='/content/drive/My Drive/wiki_zh/Model/corpusSegDone.txt'
with open(fileSegWordDonePath,'wb') as fW:
    fW.truncate(0)
    index = 0
    for path in filePath:
        LoadToSeg(path, fW)
        index += 1
        if index % 100 == 0:
            print(index)

print("Finish")

Building prefix dict from /content/drive/My Drive/wiki_zh/Model/dict.txt.big ...
Dumping model to file cache /tmp/jieba.ue8da71977d98f46d7d7945f4babe3477.cache
Loading model cost 2.294 seconds.
Prefix dict has been built successfully.


In [40]:
# jieba分詞轉word2vec向量
from gensim.models import word2vec
fileSegWordDonePath ='/content/drive/My Drive/wiki_zh/Model/corpusSegDone.txt'
fileSegWordDonePath2 ='/content/drive/My Drive/wiki_zh/Model/corpusSegDone.model'

sentences = word2vec.LineSentence(fileSegWordDonePath)
model = word2vec.Word2Vec(min_count = 1)
model.save(fileSegWordDonePath2)
print("Finish")

Finish


In [39]:
import sys
from gensim.models import word2vec 
from gensim import models

def main():   
    a = input("input:") 
    fileSegWordDonePath2 = '/content/drive/My Drive/wiki_zh/Model/corpusSegDone.model'
    model = models.Word2Vec.load(fileSegWordDonePath2)
    try:
        res = model.most_similar(a, topn=20)
        for item in res: 
            print(str(item[0]) + ':' + str(item[1]))
    except KeyError:
        print(a + " is not in the vocabulary")

if __name__=='__main__':
    main()

input:草莓


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # This is added back by InteractiveShellApp.init_path()


橙子:0.7421150803565979
藍莓:0.7363928556442261
薰衣草:0.7217570543289185
胡蘿蔔:0.7120662927627563
覆盆子:0.7095330953598022
茄子:0.7087969779968262
豌豆:0.7086479663848877
蔓越莓:0.7049437165260315
檸檬:0.7038824558258057
果醬:0.6998642086982727
菠菜:0.6995939016342163
南瓜:0.6944470405578613
薄荷:0.6873359680175781
奇異果:0.6851011514663696
蜜瓜:0.6806410551071167
仔紫背:0.6805784702301025
馬鈴薯:0.6802656650543213
花椰菜:0.6800419092178345
紅莓:0.6784149408340454
紅豆:0.6750283241271973
